In [42]:
!pip install spacy

In [43]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [44]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 20.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [45]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Google Colab is great for NLP tasks!")

In [46]:
import pdfplumber

pdf_path = "/content/HB-RESUME.pdf"

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text.strip()

resume_text = extract_text_from_pdf(pdf_path)
print(resume_text[:1000])

HIMANSHI BHARAT BANDE
Nagpur, Maharashtra | 8421959003 | bandehimanshi@gmail.com
LinkedIn: https://l1nk.dev/3zKhU | GitHub: https://github.com/hiiiimanshii
Carrier Objective
Aspiring AI/ML professional with hands-on experience in Machine Learning, NLP, and Data Analytics. Passionate about
leveraging AI to solve real-world challenges, optimize automation, and drive data-driven decision-making. Seeking
opportunities in ML engineering, data science, or AI research.
EDUCATION
B.Tech – Computer Science & Engineering (Artificial Intelligence) Expected Graduation: August 2026
G.H. Raisoni College of Engineering, Nagpur Current CGPA: 8.4
Relevant Coursework: Operating System, Deep Learning, Natural Language Processing, Computer Vision, Data
Engineering, Cloud Computing,
TECHNICAL SKILLS
Programming: Python, C++ Data Science & Analytics: Pandas, NumPy, Matplotlib, Seaborn
Databases: PostgreSQL, MySQL, DBMS Machine Learning & AI: TensorFlow, PyTorch, Scikit-learn, NLP
Web Development: HTML, CSS,

In [47]:
import re
import spacy

nlp = spacy.load("en_core_web_sm")

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.lower()  # Convert to lowercase
    return text.strip()

def preprocess_text(text):
    text = clean_text(text)
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

resume_text_cleaned = preprocess_text(resume_text)
print(resume_text_cleaned[:500])
job_description = preprocess_text("Looking for a Data Scientist with experience in Python, Machine Learning, and NLP.")

himanshi bharat bande nagpur maharashtra |   | bandehimanshi@gmail.com linkedin https://lnk.dev/zkhu | github https://github.com/hiiiimanshii carrier objective aspiring ai ml professional hand experience machine learning nlp datum analytic passionate leverage ai solve real world challenge optimize automation drive data drive decision making seek opportunity ml engineering datum science ai research education b.tech computer science engineering artificial intelligence expect graduation august   g.


In [48]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

resume_embedding = model.encode(resume_text_cleaned)
job_desc_embedding = model.encode(job_description)

similarity_score = np.dot(resume_embedding, job_desc_embedding) / (np.linalg.norm(resume_embedding) * np.linalg.norm(job_desc_embedding))

print(f"Resume Match Score: {similarity_score:.2f}")

Resume Match Score: 0.48


In [49]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    extracted_info = {
        "Name": None,
        "Email": None,
        "Phone": None,
        "Skills": []
    }

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            extracted_info["Name"] = ent.text
        elif ent.label_ == "EMAIL":
            extracted_info["Email"] = ent.text
        elif ent.label_ == "PHONE":
            extracted_info["Phone"] = ent.text

    extracted_info["Skills"] = [token.text for token in doc if token.pos_ == "NOUN"]

    return extracted_info

resume_info = extract_entities(pdf_path)
print(resume_info)

{'Name': None, 'Email': None, 'Phone': None, 'Skills': []}


In [50]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

resume_embedding = model.encode(resume_text_cleaned)
jd_embedding = model.encode(job_description)

similarity_score = np.dot(resume_embedding, jd_embedding) / (np.linalg.norm(resume_embedding) * np.linalg.norm(jd_embedding))
print(f"SBERT Similarity Score: {similarity_score:.2f}")

SBERT Similarity Score: 0.48


In [51]:
import os

def rank_resumes(resume_folder, job_description):
    scores = []
    # Get a list of all files in the current directory
    all_files = os.listdir(".")

    # Iterate over files and find PDF files
    for resume_file in all_files:
        if resume_file.endswith(".pdf"):  # Check if file is a PDF
            resume_text = extract_text_from_pdf(resume_file)  # Directly use the filename
            resume_text_cleaned = preprocess_text(resume_text)
            resume_embedding = model.encode(resume_text_cleaned)
            jd_embedding = model.encode(job_description)

            similarity_score = np.dot(resume_embedding, jd_embedding) / (np.linalg.norm(resume_embedding) * np.linalg.norm(jd_embedding))
            scores.append((resume_file, similarity_score))

    ranked_resumes = sorted(scores, key=lambda x: x[1], reverse=True)
    return ranked_resumes

job_desc = preprocess_text("Looking for a Data Scientist with experience in Python, Machine Learning, and NLP.")
ranked_results = rank_resumes(".", job_desc)  # Use "." to search in the current directory

print("\nRanked Resumes:")
for rank, (resume, score) in enumerate(ranked_results, 1):
    print(f"{rank}. {resume} - Score: {score:.2f}")


Ranked Resumes:
1. HB-RESUME.pdf - Score: 0.48
